In [1]:
import os
import re
import json
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import pickle
archivo_pickle = 'SPAM.pickle'
with open(archivo_pickle, 'rb') as file:
    # Carga los datos del archivo pickle
    spam_model = pickle.load(file)

In [4]:
class email:
    def __init__(self, id, message_id, data, subject, sender, receiver, day_week, time, time_zone, date,  answer, length, relative_path, body): #beginning, farewell,
        self._id = id
        self._message_id = message_id
        self._data = data
        
        # Verificar si 'subject' es una lista y convertirlo a cadena si es necesario
        if isinstance(subject, list):
            subject = " ".join(subject)
        
        self._subject = subject
        self._sender = sender
        self._receiver = receiver
        self._number_of_receivers = len(receiver)
        self._day_week = day_week
        self._time = time
        self._time_zone = time_zone
        self._date = date
        self._responded = False 
        self._responses = set()
        #self._beginning = beginning
        #self._farewell = farewell
        self._answer = answer
        self._length = length

        # Determinar si el correo es una respuesta y almacenar el resultado
        self._is_reply = self._subject.startswith("Re: ")
        
        self._relative_path = relative_path
        self._body = body

# Luego, al crear una instancia de 'email', esta lógica se ejecutará automáticamente


In [5]:
def email_to_dict(email):
    return email.__dict__

In [6]:
def read_email(filename, message_ids, id):
    with open(filename, 'r', errors='ignore') as file:
        data = file.read()
        id = id
        Date = r'Date: (\w+), (\d+ \w+ \d+) (\d+:\d+:\d+) (-\d{4} \(.*?\))'
        coincidencias = re.findall(Date, data)
        for day_week, date, time, time_zone in coincidencias:
            day_week = day_week
            date = date
            time = time
            time_zone = time_zone
            
        subject = re.findall(r'Subject: (.*)',data)
        
        message_id = re.search(r'Message-ID: (.*)',data)
        if message_id:
            message_id = message_id.group(1)
            
        sender_match = re.search(r'\nFrom: (.*)', data)
        if sender_match:
            sender = sender_match.group(1)
        else:
            sender = None
        
        receiver = re.findall(r'\nTo: (.*)',data)
        input_data = str(receiver)[1:-1]
        receiver_list = input_data.split(', ')
        receiver_list = [receiver.strip() for receiver in receiver_list]
        receiver_list = [s.replace("'", "") for s in receiver_list]
        receiver = [s.replace("\"", "'") for s in receiver_list]
        
        
        #beginning_pattern = (r"(?:\b(?:hello|hi|hey|good (?:morning|afternoon|evening)|greetings|dear|my dearest|hi there|hello there|hi all|hi everyone|hello everyone|dearest|yo|what's up|hey pal|yolo|howdy|how's it going|how are you|what's happening|what's new|good to see you|long time no see|it's been a while|what's cracking|how's everything))\b")
        #beginning = re.findall(beginning_pattern, data, flags=re.IGNORECASE)
        
        
        #farewell_pattern = r"(?:\b(?:Sincerely|Regards|Best regards|Kind regards|Yours sincerely|Yours faithfully|Best wishes|Take care|Warm regards|With gratitude|With best regards|Cheers|Cordially|Respectfully|Until next time|All the best|Have a great day|Talk to you soon|Looking forward to hearing from you|With thanks|Later|Catch you later|Farewell|Peace out|Stay well|All my best|Goodbye|ByeFond regards|Take it easy|So long|See you around|Keep in touch|With warm regards|With all my heart)\b)"
        #farewell = re.findall(farewell_pattern, data, flags=re.IGNORECASE)
        
        if re.search(r"-----Original Message-----", data):
            answer = 1
        elif subject[0].startswith("Re:"):
            answer = 1
        else:
            answer = 0
        
        
        body_b = data.find("X-FileName:")
        if body_b != -1:
            end_l = data.find("\n", body_b)
            if end_l != -1:
                body = data[end_l + 1:]
                words = body.split()
        length = len(words)

        relative_path = os.path.relpath(filename, start=os.path.join(os.getcwd(), '..'))
        

        #vectorizer = TfidfVectorizer(stop_words='english')
        #data_pre= vectorizer.fit_transform(data)

    
    
        if message_id in message_ids:
            return None
        else:
            email_object = email(id, message_id, data, subject, sender, receiver, day_week,time, time_zone, date, answer, length, relative_path, body) #beginning,farewell,
            return email_object
        

In [8]:
list_of_emails = []
set_of_message_ids = set()
counter = 0


# base_dir = "../maildir"
#base_dir = os.path.join(os.path.dirname(os.getcwd()), 'CODE\maildir')
dir = 'DATA\enron_mail_20150507\maildir2' 

for root, dirs, files in os.walk(dir): #
    for file in files:
        file_path = os.path.join(root, file)
        tmp_email = read_email(file_path, set_of_message_ids, counter)
        counter += 1
        if tmp_email is not None:
            list_of_emails.append(tmp_email)
            set_of_message_ids.add(tmp_email._message_id)

In [9]:
print(len(list_of_emails))
print(len(set_of_message_ids))



20524
20524


In [10]:
# Contar cuántos correos son respuestas, teniendo en cuenta que '_subject' podría ser una lista
count_replies = 0
for email in list_of_emails:
    subject = email._subject
    # Si '_subject' es una lista, unimos sus elementos en una cadena
    if isinstance(subject, list):
        subject = " ".join(subject)
    # Verificar si el asunto resultante contiene 'Re:'
    if subject.startswith("Re: "):
        count_replies += 1

# Imprimir el total de correos que son respuestas
print(f'Total de correos que son respuestas: {count_replies}')

Total de correos que son respuestas: 4523


In [11]:
def find_parent_mail(_id, _subject, _receiver, _date, _time):
    _subject = re.sub(r'Re: ', '', _subject)
    _subject = _subject.replace(' ', '')
    for email in list_of_emails:
        email_datetime_str = email._date + ' ' + email._time
        email_datetime = datetime.strptime(email_datetime_str, '%d %b %Y %H:%M:%S')
        _datetime_str = _date + ' ' + _time
        _datetime = datetime.strptime(_datetime_str, '%d %b %Y %H:%M:%S')
        if email._subject.replace(' ', '').replace('Re:', '') == _subject.replace(' ', '') and email._sender in _receiver and email_datetime < _datetime:
            email._responded = True
            email._responses.add(_id)

In [12]:
for email in list_of_emails:
    if email._is_reply:
        id = email._id
        subject = email._subject
        receiver = email._receiver
        date = email._date
        time = email._time
        find_parent_mail(id, subject, receiver, date, time)

In [13]:
for email in list_of_emails:
    email._responses = list(email._responses)

In [51]:
for counter in range(1, len(list_of_emails), 100000):
    emails_dict_list = [email_to_dict(email) for email in list_of_emails[(counter - 1):(counter + 99999)]]
    file_path = f"C:/3CD/2o cuatri/PROY III/DATA{(counter // 100000) + 1}.json"
    with open(file_path, 'w') as json_file:
        json.dump(emails_dict_list, json_file)

We create the df with the columns we want

In [14]:
import pandas as pd

# Suponemos que list_of_emails es tu lista de objetos 'email'
# Crear un DataFrame directamente desde los atributos de los objetos 'email'
data = {
    'id': [email._id for email in list_of_emails],
    'message_id': [email._message_id for email in list_of_emails],
    'subject': [email._subject for email in list_of_emails],
    'sender': [email._sender for email in list_of_emails],
    'receiver': [email._receiver for email in list_of_emails],
    'day_week': [email._day_week for email in list_of_emails],
    'time': [email._time for email in list_of_emails],
    'time_zone': [email._time_zone for email in list_of_emails],
    'date': [email._date for email in list_of_emails],
    #'beginning': [email._beginning for email in list_of_emails],
    #'farewell': [email._farewell for email in list_of_emails],
    'answer': [email._answer for email in list_of_emails],
    'length': [email._length for email in list_of_emails],
    'is_reply': [email._is_reply for email in list_of_emails],
    'responses': [list(email._responses) for email in list_of_emails],  # Asumiendo que _responses es un conjunto
    #'data':[email._data for email in list_of_emails],
    'body': [email._body for email in list_of_emails],
    'number_of_receivers':[ email._number_of_receivers for mail in list_of_emails]
}
pd.set_option('display.max_colwidth', 50)
emails_df = pd.DataFrame(data)



In [15]:
emails_df

,id,message_id,subject,sender,receiver,day_week,time,time_zone,date,answer,length,is_reply,responses,body,number_of_receivers
0,0,<29790972.1075855665306.JavaMail.evans@thyme>,"December 14, 2000 - Bear Stearns' predictions ...",1.11913372.-2@multexinvestornetwork.com,[pallen@enron.com],Wed,18:41:00,-0800 (PST),13 Dec 2000,0,1073,False,[],\nIn today's Daily Update you'll find free rep...,1
1,1,<21975671.1075855665520.JavaMail.evans@thyme>,Bloomberg Power Lines Report,messenger@ecm.bloomberg.com,[],Wed,08:35:00,-0800 (PST),13 Dec 2000,0,2205,False,[],\nHere is today's copy of Bloomberg Power Line...,1
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,Consolidated positions: Issues & To Do list Co...,phillip.allen@enron.com,"[keith.holst@enron.com, Phillip K Allen/HOU/EC...",Mon,07:16:00,-0700 (PDT),9 Oct 2000,0,901,False,[],\n---------------------- Forwarded by Phillip ...,1
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,Consolidated positions: Issues & To Do list Co...,phillip.allen@enron.com,"[keith.holst@enron.com, Phillip K Allen/HOU/EC...",Mon,07:00:00,-0700 (PDT),9 Oct 2000,0,901,False,[],\n---------------------- Forwarded by Phillip ...,1
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,,phillip.allen@enron.com,[david.delainey@enron.com],Thu,06:26:00,-0700 (PDT),5 Oct 2000,0,19,False,"[65, 1474, 1476, 87, 1118, 1119]","\nDave, \n\n Here are the names of the west de...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20519,20519,<29685494.1075840348430.JavaMail.evans@thyme>,RE: Transmission Service Agreements,kayne.coulter@enron.com,"[mitch.robinson@enron.com, john.llodra@enron.c...",Tue,08:58:13,-0800 (PST),8 Jan 2002,1,205,False,[],\n\n\n -----Original Message-----\nFrom: \tRob...,1
20520,20520,<3257673.1075840348458.JavaMail.evans@thyme>,Transmission Service Agreements,mitch.robinson@enron.com,"[john.llodra@enron.com, dale.furrow@enron.com,...",Mon,16:03:39,-0800 (PST),7 Jan 2002,0,159,False,[],\nI've incorporated Don Baughman's info into t...,1
20521,20521,<24442574.1075840348482.JavaMail.evans@thyme>,FW: transmission agreements,m..presto@enron.com,"[corry.bentley@enron.com, don.baughman@enron.com]",Mon,10:41:28,-0800 (PST),7 Jan 2002,1,82,False,[],\nDo you already have this list. Please send...,1
20522,20522,<32986178.1075840348509.JavaMail.evans@thyme>,NETCO Implemention Template,kimberly.hillis@enron.com,"[tim.belden@enron.com, f..calger@enron.com, el...",Wed,14:59:01,-0800 (PST),2 Jan 2002,0,226,False,[],\nFrom Kevin Presto:\n\nAttached is the spread...,1


### Some data preprocessing

In [22]:
# We adjust the time to the sime time_zone

def adjust_to_pst(time_str, tz_str):
    # Convert string to datetime
    time = pd.to_datetime(time_str).time()
    # Create a full datetime object with a fixed date
    datetime_obj = pd.Timestamp(f'2024-04-22 {time.hour}:{time.minute}:{time.second}')
    # Calculate offset from PST in hours
    offset = int(tz_str[:3]) + 8
    # Adjust the time
    adjusted_time = (datetime_obj - pd.Timedelta(hours=offset)).time()
    return adjusted_time
# Apply the function to adjust time to PST
emails_df['adjusted_time'] = emails_df.apply(lambda row: adjust_to_pst(row['time'], row['time_zone']), axis=1)
emails_df[['time', 'time_zone', 'adjusted_time']]



#Make 'length a discrete column
# Calcular los cortes específicos basados en 1/3 y 2/3 del rango máximo
max_value = emails_df['length'].max()
cuts = [0, max_value / 3, 2 * max_value / 3, max_value]
# Usar pd.cut para especificar los cortes manuales
emails_df['length'] = pd.cut(emails_df['length'], bins=cuts, labels=['short', 'medium', 'large'], include_lowest=True)

emails_df

,id,message_id,subject,sender,receiver,day_week,time,time_zone,date,answer,length,is_reply,responses,body,number_of_receivers,adjusted_time,category_custom
0,0,<29790972.1075855665306.JavaMail.evans@thyme>,"December 14, 2000 - Bear Stearns' predictions ...",1.11913372.-2@multexinvestornetwork.com,[pallen@enron.com],Wed,18:41:00,-0800 (PST),13 Dec 2000,0,short,False,[],\nIn today's Daily Update you'll find free rep...,1,18:41:00,short
1,1,<21975671.1075855665520.JavaMail.evans@thyme>,Bloomberg Power Lines Report,messenger@ecm.bloomberg.com,[],Wed,08:35:00,-0800 (PST),13 Dec 2000,0,short,False,[],\nHere is today's copy of Bloomberg Power Line...,1,08:35:00,short
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,Consolidated positions: Issues & To Do list Co...,phillip.allen@enron.com,"[keith.holst@enron.com, Phillip K Allen/HOU/EC...",Mon,07:16:00,-0700 (PDT),9 Oct 2000,0,short,False,[],\n---------------------- Forwarded by Phillip ...,1,06:16:00,short
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,Consolidated positions: Issues & To Do list Co...,phillip.allen@enron.com,"[keith.holst@enron.com, Phillip K Allen/HOU/EC...",Mon,07:00:00,-0700 (PDT),9 Oct 2000,0,short,False,[],\n---------------------- Forwarded by Phillip ...,1,06:00:00,short
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,,phillip.allen@enron.com,[david.delainey@enron.com],Thu,06:26:00,-0700 (PDT),5 Oct 2000,0,short,False,"[65, 1474, 1476, 87, 1118, 1119]","\nDave, \n\n Here are the names of the west de...",1,05:26:00,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20519,20519,<29685494.1075840348430.JavaMail.evans@thyme>,RE: Transmission Service Agreements,kayne.coulter@enron.com,"[mitch.robinson@enron.com, john.llodra@enron.c...",Tue,08:58:13,-0800 (PST),8 Jan 2002,1,short,False,[],\n\n\n -----Original Message-----\nFrom: \tRob...,1,08:58:13,short
20520,20520,<3257673.1075840348458.JavaMail.evans@thyme>,Transmission Service Agreements,mitch.robinson@enron.com,"[john.llodra@enron.com, dale.furrow@enron.com,...",Mon,16:03:39,-0800 (PST),7 Jan 2002,0,short,False,[],\nI've incorporated Don Baughman's info into t...,1,16:03:39,short
20521,20521,<24442574.1075840348482.JavaMail.evans@thyme>,FW: transmission agreements,m..presto@enron.com,"[corry.bentley@enron.com, don.baughman@enron.com]",Mon,10:41:28,-0800 (PST),7 Jan 2002,1,short,False,[],\nDo you already have this list. Please send...,1,10:41:28,short
20522,20522,<32986178.1075840348509.JavaMail.evans@thyme>,NETCO Implemention Template,kimberly.hillis@enron.com,"[tim.belden@enron.com, f..calger@enron.com, el...",Wed,14:59:01,-0800 (PST),2 Jan 2002,0,short,False,[],\nFrom Kevin Presto:\n\nAttached is the spread...,1,14:59:01,short


We save the final df on a json to use for the static model

In [24]:
emails_df.to_json('emails.json',orient='records', lines=True)